<a href="https://colab.research.google.com/github/37stu37/GColab/blob/main/Practical_3_Natural_hazard_impact_%2C_exposure_and_vulnerability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Practical 3: Natural hazard impact , exposure and vulnerability

## Introduction
In this practical, you will model the impacts that are expected to result if the 1934 Mw 8.3 Nepal-Bihar earthquake were to (re-)occur today. You will use shaking data and population data provided to analyse the number of fatalities expected and compare the frequency-magnitude distributions to impacts from the 2015 Gorkha earthquake in Nepal.

##Aims
1. To work with essential GIS data into Python (Google Colab)
2. To extract statistics from GIS raster layers
3. To calculate the probability of building damage state 
4. To estimate fatalities from a disaster 
5. To create comparative study and reflect on the uncertainty and limitations of your data

##Format of the assignment
You need to write a summary of your findings in a short scientific abstract of no more than 200 words. The abstract should provide a thorough summary of the work undertaken and should be well-written and organised. 

The abstract / executive summary should provide:

- An understanding of the fatality numbers from the 1934 and 2015 earthquake. How does it relate to building damages?
- Based on the data you have, a comparative study of the two events. How do the data from the 2015 earthquake compare to the 1934 earthquake and what does this tell you about both events?
- Explain the limitations of the current assumptions. What is the effect of those assumptions and how is this likely to be wrong? What impact could this have on your scenario results? 

##Submitting the assignment
You will answer the questions and write your free form abstract in a separate word document / text file (graphs and maps are welcome to illustrate your point) that must be 200 words maximum. The document must be saved as a pdf and uploaded to Turnitin.

The assignment is due by **7th February 2023 4pm**.

<br/><br/>

❗ <font color='red'>**Before going any further - Please save the notebook in your OWN Google Drive**</font> ❗


<br/><br/>



## Getting started

First, you need to install the packages using pip.
the *!* is used by Google Colab to understand that it needs to run a shell command

In [ ]:
%%capture
!pip install rioxarray
!pip install geopandas
!pip install rasterstats
!pip install mapclassify
!pip install plotly
!pip install "notebook>=5.3" "ipywidgets>=7.5"

Second, you need to load modules. The *import* tag install a library that can now be used in your script

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rioxarray as rio
import plotly.express as px

px.defaults.template = "ggplot2"

Before you begin, please read through the entire practical exercise. You may not understand everything, but you will have a better sense of what is expected of you and what you will be doing.

## **Part I: Loading the data sets**

### Access files in Goolge Colab 

---

**How?**

Access local files through the file-explorer
Uploading files from local file system through file-explorer
You can either use the upload option at the top of the file-explorer pane to upload any file(s) from your local file system to Colab in the present working directory. 

To upload files directly to a subdirectory you need to:

    1. Click on the three dots visible when you hover above the directory 

    2. Select the “upload” option.

<br/>

**Now that you know how to load files we will download the data and load them in Google Colab:**

<br/>

1. The Data folder is available in Blackboard. Copy each of the files to your disk

2. The first step in scenario development is to model the ground shaking that will result from your chosen earthquake and compare it to population distribution. For this exercise I have modelled the shaking for the 1934 Mw 8.0 Nepal-Bihar earthquake for you (***PGA_1934EQ.tif***). I have also given you a shapefile of Nepali Village Development Committees (VDCs – similar to Local Authorities) that contain details on population and building type (***Nepal_VDCs.shp***). Note that Nepal’s local government system has changed recently, but we will use VDC-level data from the census in 2011. You have a also a csv file of the Fatalities per VDC after the 2015 Gorkha earthquake (***2015_EQData.csv***)

3. Load those dataset in Google Colab as explain earlier (it will only remain temporarily on your drive)

### Access files in QGIS

---

Load the same datasets in QGIS following the subsequent steps:


<pre>
Click Layer
    &darr; 
Add Layer
    &darr; 
Add Vector (.shp)
Add Raster (.tif)
Add Delimited Text Layer (.csv)
</pre>


The attribute table of the ***Nepal_VDCs.shp*** contains the name and a unique code for each VDC along with population data and the total number of different building types within that VDC. The column titles refer to the following:

    WDN – Wooden (Bamboo or Timber)
    RCNE – Non-Engineered Reinforced Concrete
    BCF – Flexible Brick & Concrete
    BCR – Rigid Brick & Concrete
    SM – Stone & Mud


The ***PGA_1934EQ.tif*** file is estimate of the peak ground accelerations from the 1934 earthquake, expressed as a fraction of the Earth’s gravitational acceleration, g. These are not observed values, but have been estimated using the OpenSHA model. You may wish to better visualise the shaking by playing with the Symbology of the PGA_1934EQ file in QGIS. We will also display all the data loaded in this notebook.

### Display the data in Google Colab

---


Display the .tif raster file using rasterio/rioxarray library

Rasterio and Rioxarray are Python options (not the only ones) for accessing the many different kind of raster data files used in the GIS field

In [ ]:
PGA_1934EQ = rio.open_rasterio('PGA_1934EQ.tif', masked=True)

PGA_1934EQ

In [ ]:
print("\n Ground motion filed (PGA) of the 1934 earthquake \n")
PGA_1934EQ.squeeze().plot.imshow();

There are different options to display csv and text files in Google Colab

 - using the Magics (%) capability of Google Colab (not the preferred approach)

In [ ]:
%cat 2015_EQData.csv

 - or the (very popular) Pandas library 

Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language

In [ ]:
# using pandas library
EQDdata = pd.read_csv("2015_EQData.csv")
display(EQDdata.head(5)) # head is used to select only the first n lines - here 5 lines with .head(5)

Then the shapefile can be open using the Geopandas library

In [ ]:
nepal_VDCs = gpd.read_file("Nepal_VDCs.shp")

nepal_VDCs

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1, figsize=(10,15))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

# the column argument will be used to color code the display
nepal_VDCs.plot(column='Pop_Total', legend=True, ax=ax, cax=cax);

In [ ]:
# use the scheme argument to display a quantile classification
nepal_VDCs.plot(column='Pop_Total', scheme='quantiles', legend=True, figsize=(10,15));

## Part II: Damage modelling

In order to estimate the number of building collapses, first you need to assess the average shaking each VDC experiences in the scenario. Once you have estimated this, you will export the data to CSV to perform some modelling calculations.

**IN QGIS:**

QGIS can calculate the average values from a raster file within different zones by using a tool called Zonal Statistics. You will need to use Zonal Statistics to calculate the mean 1934 shaking within each VDC following these steps:


<pre>
Click Processing Toolbox
          &darr; 
Zonal Statistics
  &rarr; Input layer - VDC shapefile
  &rarr; Raster layer - Earthquake raster
          &darr; 
Remove "_" in the output column prefix
          &darr; 
Calculate the Mean
          &darr; 
Run and Save the output Shapefile
          &darr; 
</pre>




Once the calculation has successfully completed, your new file should appear in the Layers. Right-click the file and select Open Attribute Table. 

To check the calculation has completed accurately, first make sure that the values in the Mean column appear sensible – remember, these are earthquake accelerations as a fraction of g.

Then we need to export Stats to Google Colab:

<pre>
Right click on the output Shapefile file
Export:
  &rarr; Fromat = CSV
  &rarr; CRS = EPSG 32645 - WGS84 / UTM Zone 45N
  &rarr; Filename = "stats.csv"
  </pre>

You can now upload the ***stats.csv*** in Google Colab as you have done previously.

And read it using *pd.read_csv*:



In [ ]:
vdc_stats = pd.read_csv("stats.csv")
display(vdc_stats)

We can calculate the total building count by adding up the buildings of each type

In [ ]:
vdc_stats["total building count"] = vdc_stats["WDN"] + vdc_stats["RCNE"] + vdc_stats["BCF"] + vdc_stats["BCR"] + vdc_stats["SM"]
display(vdc_stats)

Now you need to model building damage. To do this, you need to know how different types of buildings perform under shaking of different strengths.

Empirical data from global earthquakes have been used to derive fragility curves for each of the building types in your data set. Below is a set of fragility curves that describe the rate of complete damage for each type. 

In this instance, *complete damage* refers to a state in which the structure has collapsed or is in imminent danger of collapse; all structural components (walls, floors, roofs etc.) are damaged and the building will need to be demolished and rebuilt entirely.

As a reminder, here are the typologies in the VDC attributes

<pre>
*   WDN - Wood frame
*   RCNE - Reinforce Concrete
*   BCF - Brick Flexible
*   BCR - Brick Rigid
*   SM - Stone & Mud
</pre>

The cumulative distribution of two-parameter lognormal distribution functions(mean and standard deviation) are used to represent the fragility curves.
https://en.wikipedia.org/wiki/Log-normal_distribution


We will use the lognorm module from scipy.stats library to sample the probability of complete damage state for a specific PGA(g) value of shaking

In [ ]:
from scipy.stats import lognorm
import matplotlib.pyplot as plt
import numpy as np

# get 100 artificial pga values between 0 and 3g
pgas = np.linspace(0,3, 100)

# two parameters - mean and standard deviation - of the lognormal fragility functions
# for the different building typologies
###########
WDN_mean = 1.6446
WDN_std = 1.1701

RCNE_mean = 1.29
RCNE_std = 0.83

BCF_mean = 0.39
BCF_std = 0.69

BCR_mean = 1.26
BCR_std = 1.96

SM_mean = 0.23
SM_std = 0.31
############

# Create the fragility curves for each typologies
F_WDN = [lognorm(WDN_std,scale=WDN_mean).cdf(p) for p in pgas]
F_RCNE = [lognorm(RCNE_std,scale=RCNE_mean).cdf(p) for p in pgas]
F_BCF = [lognorm(BCF_std,scale=BCF_mean).cdf(p) for p in pgas]
F_BCR = [lognorm(BCR_std,scale=BCR_mean).cdf(p) for p in pgas]
F_SM = [lognorm(SM_std,scale=SM_mean).cdf(p) for p in pgas]

# Plot the fragility functions
alf=1
fig, axs = plt.subplots(1, 1, figsize=(5,3))

axs.plot(pgas, F_WDN, c='blue', label="WDN", alpha=alf)
axs.plot(pgas, F_RCNE, c='lightblue', label="RCNE", alpha=alf)
axs.plot(pgas, F_BCF, c='orange', label="BCF", alpha=alf)
axs.plot(pgas, F_BCR, c='lightgreen', label="BCR", alpha=alf)
axs.plot(pgas, F_SM, c='red', label="SM", alpha=alf)


axs.set(ylabel="Probability of complete damage", xlabel="PGA (g)")
axs.set_ylim(0,1.05)
axs.legend()
axs.legend(loc='lower right', frameon=True)
plt.show()


We then need, for each typologies, to calculate the probability of complete damage for the pga within the VDC

In [ ]:
# calculate the fragility of each building types for each pgas
pgas = vdc_stats["mean"].values

Complete_perc_WDN = [lognorm(WDN_std,scale=WDN_mean).cdf(p) for p in pgas]
Complete_perc_RCNE = [lognorm(RCNE_std,scale=RCNE_mean).cdf(p) for p in pgas]
Complete_perc_BCF = [lognorm(BCF_std,scale=BCF_mean).cdf(p) for p in pgas]
Complete_perc_BCR = [lognorm(BCR_std,scale=BCR_mean).cdf(p) for p in pgas]
Complete_perc_SM = [lognorm(SM_std,scale=SM_mean).cdf(p) for p in pgas]

- Now you can calculate the number of buildings of each type, in each VDC, that are expected to suffer complete damage.

- We can then calculate the total number of building in a *Complete* damage state (we round the number as decimal in building number count doesn't really make sense)

 



In [ ]:
# calculate the actual number of building predicted to be in a "Complete damage state"

Complete_bldg_WDN = Complete_perc_WDN * vdc_stats["WDN"].values
Complete_bldg_RCNE = Complete_perc_RCNE * vdc_stats["RCNE"].values
Complete_bldg_BCF = Complete_perc_BCF * vdc_stats["BCF"].values
Complete_bldg_BCR = Complete_perc_BCR * vdc_stats["BCR"].values
Complete_bldg_SM = Complete_perc_SM * vdc_stats["SM"].values

vdc_stats["Complete_bldg_total"] = Complete_bldg_WDN + Complete_bldg_RCNE + Complete_bldg_BCF + Complete_bldg_BCR + Complete_bldg_SM
vdc_stats["Complete_bldg_total"] = [round(x) for x in vdc_stats["Complete_bldg_total"]]

vdc_stats

Typically, earthquake fatalities predominantly occur in buildings that collapse. 

You therefore need to estimate how many of your completely damaged buildings will fully collapse and thus how many buildings are expected to cause fatalities. 

Again, global empirical data for this has been collected and shows that different building types are more likely to collapse than others:

| Building Type      | Probability that completely damaged building will collapse |
| ----------- | ----------- |
| WDN      | 3%       |
| RCNE   | 13%        |
| BCF      | 15%       |
| BCR   | 15%        |
| SM   | 15%        |

- We can calculate the number of actual collapse by multiplying the number of building in a Complete damage state with the probability of actual collapse

- We can then add the results from all the collapse building to get a total per VDC

In [ ]:
Collapse_bldg_WDN = Complete_bldg_WDN * 0.03
Collapse_bldg_RCNE = Complete_bldg_RCNE * 0.13
Collapse_bldg_BCF = Complete_bldg_BCF * 0.15
Collapse_bldg_BCR = Complete_bldg_BCR * 0.15
Collapse_bldg_SM = Complete_bldg_SM * 0.15

vdc_stats["Collapse_bldg_total"] = Collapse_bldg_WDN + Collapse_bldg_RCNE + Collapse_bldg_BCF + Collapse_bldg_BCR + Collapse_bldg_SM
vdc_stats["Collapse_bldg_total"] = [round(x) for x in vdc_stats["Collapse_bldg_total"]]

vdc_stats

We can then plot the total numbers of building suffering complete damage as well as those that collapse. Compare the results.

In [ ]:
# import seaborn as sns

barWidth = 0.2
fig = plt.subplots(figsize =(3, 5))

# Set position of bar on X axis
br1 = 1
 
# Make the plot
plt.bar(br1, np.sum(vdc_stats["total building count"].values), color ='white', edgecolor ='grey', label ='Total building count')
plt.bar(br1, np.sum(vdc_stats["Complete_bldg_total"].values), color ='b', edgecolor ='grey', label ='Complete damage state')
plt.bar(br1, np.sum(vdc_stats["Collapse_bldg_total"].values), color ='r', edgecolor ='grey', label ='Collapse')
 
plt.ylabel('Count', fontweight ='bold', fontsize = 15)
 
plt.legend()
plt.show()

## Part III: Fatality modelling

Now that you have an estimate of the number of buildings collapsing, you can start to model how many people might be killed in each VDC. For this exercise, I want you to assume a night-time scenario, where everybody is indoors at the time of the earthquake.

First, you need to estimate how many people in each VDC live in collapsed buildings. Without any other knowledge, you should assume that the population in each VDC is equally distributed between all types of buildings. You therefore need to divide the total population in each VDC by the total number of buildings to calculate the average number of people per building.

Let's create a new column *Pop_per_house* in the dataset

In [ ]:
vdc_stats["Pop_per_house"] = vdc_stats["Pop_Total"] / vdc_stats["total building count"]

display(vdc_stats)


Now you can model the number of fatalities using Fatality Rates. The number of people killed by collapsing buildings varies by building type – buildings comprised of heavier material kill more people than buildings comprised of light material. Using the Fatality Rates provided below, calculate the total number of people killed in each building type in columns, then sum up to find the total number of fatalities in each VDC.

| Building Type      | Fatality Rate (%) |
| ----------- | ----------- |
| WDN      | 0.5%       |
| RCNE   | 10%        |
| BCF      | 5%       |
| BCR   | 15%        |
| SM   | 5%        |

In [ ]:
# Collapse building * Falality Rate

Fatality_bldg_WDN = Collapse_bldg_WDN * 0.005
Fatality_bldg_RCNE = Collapse_bldg_RCNE * 0.10
Fatality_bldg_BCF = Collapse_bldg_BCF * 0.05
Fatality_bldg_BCR = Collapse_bldg_BCR * 0.15
Fatality_bldg_SM = Collapse_bldg_SM * 0.05

vdc_stats["Fatalities"] = Fatality_bldg_WDN + Fatality_bldg_RCNE + Fatality_bldg_BCF + Fatality_bldg_BCR + Fatality_bldg_SM
vdc_stats["Fatalities"] = [round(x) for x in vdc_stats["Fatalities"]]


display(vdc_stats)

Plot the total number of fatalities per building type. We use the plotly library to get interactive plots (feel free to try others libraries if you wish)

In [ ]:
import plotly.graph_objects as go

Total_Fatalities = np.sum(vdc_stats.Fatalities)

fig = go.Figure()

T = ['WDN', 'RCNE', 'BCF', 'BCR', 'SM']
F = [np.sum(Fatality_bldg_WDN), np.sum(Fatality_bldg_RCNE), np.sum(Fatality_bldg_BCF),
     np.sum(Fatality_bldg_BCR), np.sum(Fatality_bldg_SM)]


# Add traces
fig = go.Figure(data=[go.Bar(x=T, y=F)])
fig.update_layout(title_text='Fatalities by Building typologies')

fig.show()

## Part IV: Distributions and comparisons with past events

The 2015 Gorkha earthquake occurred on 25 April 2015 at 11:56 am NST (06:11:26 UTC) at a depth of approximately 8.2 km (5.1 mi) (which is considered shallow and therefore more damaging than quakes that originate deeper in the ground), with its epicentre approximately 34 km (21 mi) east-southeast of Lamjung, Nepal, lasting approximately fifty seconds. The earthquake was initially reported as 7.5 Mw by the United States Geological Survey (USGS) before it was quickly upgraded to 7.8 Mw

Using the step and data that you are now familiar with and the data from *2015_EQData.csv* showing the fatalities that would have resulted from the 2015 Gorkha earthquake if it had happened at night-time. The results suggest a total of ~36,000 people would have been killed, ~27,000 more than were actually killed by the daytime event.

Here are some functions that could prove useful for your assignment.

```
# Load 2015 data
eq2015 = pd.read_csv("2015_EQData.csv")

# merging two dataset by using:
merge = vdc_stats.merge(eq2015, left_on='VDC_NAME', right_on='VDC')

```

Please refer to the beginning of this notebook for the assignment tasks. Feel free to use your notebook or QGIS to create maps and plots (you can use screenshots to easily copy to your external document).
